In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [2]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
from quetzal.io import excel
import numpy as np
import pandas as pd
import geopandas as gpd
from syspy.skims import skims

SQLalchemy is not installed. No support for SQL output.


In [3]:
training_folder = '../../'
plot_path = training_folder + 'outputs/plot/'

In [4]:
sm = stepmodel.read_zippedpickles(training_folder + 'model/{scen}/aon_pathfinder'.format(scen=scenario))
distribution = stepmodel.read_zippedpickles(training_folder + 'model/{scen}/distribution'.format(scen=scenario))

zones: 100%|█████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 31.68it/s]


In [5]:
walk_speed = 4

var = excel.read_var(file='../../inputs/parameters.xlsx', scenario=scenario)
boarding_time = var['general']['boarding_time']
var

category            parameter        
general             description          base
                    parent               base
                    bus_speed_factor        1
                    boarding_cost_acf     300
                    boarding_cost_wor     300
                                         ... 
vehicle_capacities  brt                   160
                    bus                    90
                    small                  20
                    local                  20
                    bajaj                   5
Name: base, Length: 69, dtype: object

## Pathfinders per mode

### Bajaj lines

In [6]:
pt = sm.copy()
pt.pt_los = sm.pt_los
pt.pt_los.set_index(['origin', 'destination'])

gtime pathfinder_session
origin   destination                                      
k2_papa  k2_papa               0.000000   optimal_strategy
         k4_highschool      2369.555347   optimal_strategy
         k9_gende_ticharo   3625.244703   optimal_strategy
         zone_152           9036.410748   optimal_strategy
         k8_chaka           3539.028989   optimal_strategy
...                                 ...                ...
zone_135 k1_genede_buta     7397.352683   optimal_strategy
         k1_genede_amerach  6415.678792   optimal_strategy
         k5_tekiliye        6581.337105   optimal_strategy
         k7_seariea         5573.327269   optimal_strategy
         zone_135              0.000000   optimal_strategy

[40000 rows x 2 columns]

### Walk

In [7]:
walk = sm.copy()
walk.road_links['time'] = walk.road_links['walk_time']
walk.step_road_pathfinder(all_or_nothing = True)
walk.car_los['true_time'] = walk.car_los['time'] 
walk.car_los.set_index(['origin', 'destination'])

time  \
origin            destination                       
k1_abdulahi_mussa k1_abdulahi_mussa      0.000000   
                  k1_agedalis         1508.216761   
                  k1_gender_reye      2226.324209   
                  k1_genede_amerach   1063.786651   
                  k1_genede_buta       654.351057   
...                                           ...   
zone_214          zone_210           19741.066004   
                  zone_211           20528.071497   
                  zone_212            3659.998081   
                  zone_213            1852.706785   
                  zone_214               0.000000   

                                                                                  path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                [k1_abdulahi_mussa]   
                  k1_agedalis        [k1_abdulahi_mussa, road_node_249, road_node_4...   
                  k1_gender_reye     [k1_abdulahi_mussa, road_node_249, road_node_4...   
                  k1_genede_amerach  [k1_abdulahi_mussa, road_node_249, road_node_4...   
                  k1_genede_buta     [k1_abdulahi_mussa, road_node_4513, road_node_...   
...                                                                                ...   
zone_214          zone_210           [zone_214, road_node_1396, road_node_1824, roa...   
                  zone_211           [zone_214, road_node_1396, road_node_1824, roa...   
                  zone_212           [zone_214, road_node_3458, road_node_3590, roa...   
                  zone_213           [zone_214, road_node_1396, road_node_1824, zon...   
                  zone_214                                                  [zone_214]   

                                                                             node_path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                                 []   
                  k1_agedalis        [road_node_249, road_node_4327, road_node_4619...   
                  k1_gender_reye     [road_node_249, road_node_4327, road_node_4624...   
                  k1_genede_amerach  [road_node_249, road_node_4327, road_node_4624...   
                  k1_genede_buta     [road_node_4513, road_node_4424, road_node_212...   
...                                                                                ...   
zone_214          zone_210           [road_node_1396, road_node_1824, road_node_181...   
                  zone_211           [road_node_1396, road_node_1824, road_node_181...   
                  zone_212            [road_node_3458, road_node_3590, road_node_2716]   
                  zone_213                            [road_node_1396, road_node_1824]   
                  zone_214                                                          []   

                                                                             link_path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                                 []   
                  k1_agedalis        [road_link_3597, road_link_2360, road_link_149...   
                  k1_gender_reye     [road_link_3597, road_link_2359, road_link_114...   
                  k1_genede_amerach   [road_link_3597, road_link_2359, road_link_3756]   
                  k1_genede_buta     [road_link_11485, road_link_11502, road_link_1...   
...                                                                                ...   
zone_214          zone_210           [road_link_11771, road_link_3953, road_link_39...   
                  zone_211           [road_link_11771, road_link_3953, road_link_39...   
                  zone_212                           [road_link_15283, road_link_7869]   
                  zone_

### Private (bajaj contract)

In [8]:
priv = sm.copy()
priv.step_road_pathfinder(all_or_nothing = True)
priv.car_los.set_index(['origin', 'destination'])

time  \
origin            destination                      
k1_abdulahi_mussa k1_abdulahi_mussa     0.000000   
                  k1_agedalis         586.990737   
                  k1_gender_reye      600.373819   
                  k1_genede_amerach   338.111971   
                  k1_genede_buta      195.522113   
...                                          ...   
zone_214          zone_210           3262.132351   
                  zone_211           3463.163647   
                  zone_212           1122.605043   
                  zone_213            790.509786   
                  zone_214              0.000000   

                                                                                  path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                [k1_abdulahi_mussa]   
                  k1_agedalis        [k1_abdulahi_mussa, road_node_5167, road_node_...   
                  k1_gender_reye     [k1_abdulahi_mussa, road_node_5167, road_node_...   
                  k1_genede_amerach  [k1_abdulahi_mussa, road_node_5167, road_node_...   
                  k1_genede_buta     [k1_abdulahi_mussa, road_node_5167, road_node_...   
...                                                                                ...   
zone_214          zone_210           [zone_214, road_node_3458, road_node_3112, roa...   
                  zone_211           [zone_214, road_node_3458, road_node_3112, roa...   
                  zone_212           [zone_214, road_node_3458, road_node_3590, roa...   
                  zone_213           [zone_214, road_node_3458, road_node_3112, roa...   
                  zone_214                                                  [zone_214]   

                                                                             node_path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                                 []   
                  k1_agedalis        [road_node_5167, road_node_4887, road_node_249...   
                  k1_gender_reye     [road_node_5167, road_node_4887, road_node_249...   
                  k1_genede_amerach  [road_node_5167, road_node_4887, road_node_249...   
                  k1_genede_buta     [road_node_5167, road_node_4513, road_node_442...   
...                                                                                ...   
zone_214          zone_210           [road_node_3458, road_node_3112, road_node_139...   
                  zone_211           [road_node_3458, road_node_3112, road_node_139...   
                  zone_212            [road_node_3458, road_node_3590, road_node_2716]   
                  zone_213           [road_node_3458, road_node_3112, road_node_139...   
                  zone_214                                                          []   

                                                                             link_path  \
origin            destination                                                            
k1_abdulahi_mussa k1_abdulahi_mussa                                                 []   
                  k1_agedalis        [road_link_3620, road_link_3610, road_link_359...   
                  k1_gender_reye     [road_link_3620, road_link_3610, road_link_359...   
                  k1_genede_amerach  [road_link_3620, road_link_3610, road_link_359...   
                  k1_genede_buta     [road_link_3621, road_link_11485, road_link_11...   
...                                                                                ...   
zone_214          zone_210           [road_link_15004, road_link_8017, road_link_11...   
                  zone_211           [road_link_15004, road_link_8017, road_link_11...   
                  zone_212                           [road_link_15283, road_link_7869]   
                  zone_213         

## Concatenation

In [9]:
#Add relevant columns to path dataframes

pt.pt_los['route_type']='pt'
pt.road_links['walk_time'] = sm.road_links['length'] / 1000 / walk_speed * 3600
pt.analysis_strategy_time(walk_on_road=True, boarding_time=boarding_time)
pt.pt_los['waiting_time'] = np.round(pt.pt_los['waiting_time'])

pt.pt_los['true_time'] = pt.pt_los['time']
priv.car_los['true_time'] = priv.car_los['time']
walk.car_los['true_time'] = walk.car_los['time'] * float(var['speed']['walk_on_road']) / walk_speed

walk.car_los['route_type']='walk'
priv.car_los['route_type']='car'

pt.pt_los['route_types']=[('pt',) for element in pt.pt_los.index]
walk.car_los['route_types']=[('walk',) for element in walk.car_los.index]
priv.car_los['route_types']=[('car',) for element in priv.car_los.index]

pt.pt_los['price']=var['fare']['pt']
walk.car_los['price']=0
priv.car_los['price']=var['fare']['car']
pt.pt_los['ntransfers']=0 #on supose 0
walk.car_los['ntransfers']=0
priv.car_los['ntransfers']=0

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:10<00:00, 19.93it/s]


In [10]:
#concatenate all modes los
pt.pt_los['time'] = pt.pt_los['gtime']
pathfound = sm.copy()
pathfound.los = pd.concat(
    [walk.car_los, pt.pt_los])
pathfound.los['time'] = np.round(pathfound.los['time'], 2)
pathfound.los.drop_duplicates(subset=['origin', 'destination', 'time'], inplace=True)
# On supprime les itinéraires «TC» qui en fait sont de la marche à pied
pathfound.los = pd.concat([pathfound.los, priv.car_los])
pathfound.los.reset_index(drop=True, inplace=True) #unique index

## Logit scales

In [11]:
kwargs = var['preparation_logit'].to_dict()
pathfound.preparation_logit(**kwargs, segments=['todos'])
pathfound.segments=['todos']

In [12]:
pathfound.links

,a,b,link_sequence,direction_id,line,trip_id,route_id,road_a,road_b,road_node_list,...,road_length,geometry,route_type,mode,headway,vehicle_capacity,per_hour,capacity,length,time
link_0,university,eshet grocery,0,0,0,0,0,road_node_797,road_node_1842,"[road_node_797, road_node_1544, road_node_4944...",...,998.098963,"LINESTRING (811339.957 1064005.306, 811356.936...",bajaj,bajaj,180,5,20.0,100,881.078155,166.523771
link_1,eshet grocery,seido,1,0,0,0,0,road_node_1842,road_node_1667,"[road_node_1842, road_node_1492, road_node_455...",...,788.665218,"LINESTRING (811757.899 1063351.698, 811713.919...",bajaj,bajaj,180,5,20.0,100,788.665218,149.057726
link_2,seido,eshet grocery,0,0,0_bis,0_bis,0_bis,road_node_1667,road_node_1842,"[road_node_1667, road_node_5020, road_node_219...",...,788.665218,"LINESTRING (811502.112 1062605.703, 811505.421...",bajaj,bajaj,180,5,20.0,100,788.665218,149.057726
link_3,eshet grocery,university,1,0,0_bis,0_bis,0_bis,road_node_1842,road_node_797,"[road_node_1842, road_node_3937, road_node_110...",...,995.328246,"LINESTRING (811757.899 1063351.698, 811761.929...",bajaj,bajaj,180,5,20.0,100,879.443209,166.214767
link_4,university,eshet grocery,0,0,1,1,1,road_node_797,road_node_1842,"[road_node_797, road_node_1544, road_node_4944...",...,998.098963,"LINESTRING (811339.957 1064005.306, 811356.936...",bajaj,bajaj,180,5,20.0,100,881.078155,166.523771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
link_1523,sabian agip,bridge,2,0,9_bis,9_bis,9_bis,road_node_822,road_node_4673,"[road_node_822, road_node_4075, road_node_4100...",...,340.489477,"LINESTRING (812218.296 1062355.854, 812330.191...",bajaj,bajaj,180,5,20.0,100,340.489477,64.352511
link_1524,bridge,dipo,3,0,9_bis,9_bis,9_bis,road_node_4673,road_node_2820,"[road_node_4673, road_node_2546, road_node_368...",...,715.037442,"LINESTRING (812535.301 1062231.921, 812568.582...",bajaj,bajaj,180,5,20.0,100,715.037442,135.142077
link_1525,dipo,sholla hotel,4,0,9_bis,9_bis,9_bis,road_node_2820,road_node_1283,"[road_node_2820, road_node_2829, road_node_170...",...,364.975454,"LINESTRING (812980.946 1061702.262, 812991.756...",bajaj,bajaj,180,5,20.0,100,364.975454,68.980361
link_1526,sholla hotel,karamara,5,0,9_bis,9_bis,9_bis,road_node_1283,road_node_2484,"[road_node_1283, road_node_1053, road_node_420...",...,928.906495,"LINESTRING (813176.167 1061416.487, 813156.024...",bajaj,bajaj,180,5,20.0,100,774.088746,146.302773


In [13]:
pathfound.utility_values.T

value,time,price,ntransfers,mode_utility
segment,,,,
root,-0.00055,-0.099,-0.25,1.0
todos,-0.00055,-0.099,-0.25,1.0


In [14]:
pathfound.logit_scales.drop('bajaj', inplace=True, errors='ignore')
pathfound.logit_scales.T

route_type,car,root,walk,pt
segment,,,,
root,0.0,1.0,0.0,0.5
todos,0.0,1.0,0.0,0.5


In [15]:
pathfound.mode_utility.loc['pt']=0
pathfound.mode_utility.loc['pt','todos']=var['mode_utility']['pt']
pathfound.mode_utility.loc['car','todos']=var['mode_utility']['car']
pathfound.mode_utility.loc['walk','todos']=var['mode_utility']['walk']

pathfound.mode_utility.T

route_type,car,root,bajaj,walk,pt
segment,,,,,
root,0,0,0,0,0
todos,0,0,0,0,0


In [16]:
pathfound.mode_nests.drop('bajaj', inplace=True, errors='ignore')

## Utility

In [17]:
# Calculate the utilities and the probabilities

pathfound.analysis_mode_utility()
#segment='all'
#pathfound.los[(segment, 'utility')]=0
pathfound.initialize_logit()
pathfound.step_logit()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.80it/s]


In [18]:
pathfound.los['gtime'] = pathfound.los['time']
pathfound.los['time'] = pathfound.los['true_time']

# ASSIGNMENT

In [19]:
pathfound.volumes = distribution.volumes.copy()
pathfound.volumes = pathfound.volumes.rename(columns = {'pt_captive':'todos'})
pathfound.compute_los_volume()

# TOZIPPEDPICKLES

In [20]:
pathfound.to_zippedpickles(
    training_folder + 'model/{scen}/modal_split'.format(scen=scenario), 
)

volumes: 100%|█████████████████████████████████████████████████████████████████████████| 45/45 [00:09<00:00,  4.58it/s]


In [ ]:
end_of_notebook